In [112]:
import pandas as pd

In [113]:
def calc_delta_m(base_acc_nap: float, base_error_ntp: float, base_error_rtp: float, acc_nap: float, error_ntp: float, error_rtp: float):
    """Calculates delta-M metric
    """

    tasks = []
    if acc_nap is not None and not pd.isna(acc_nap):
        tasks.append(acc_nap)
    if error_ntp is not None and not pd.isna(error_ntp):
        tasks.append(error_ntp)
    if error_rtp is not None and not pd.isna(error_rtp):
        tasks.append(error_rtp)

    delta_m = (
        1 / len(tasks)
        * (
            ((-1) * (acc_nap - base_acc_nap) / base_acc_nap if acc_nap is not None and not pd.isna(acc_nap) else 0)
            + (1 * (error_ntp - base_error_ntp) / base_error_ntp if error_ntp is not None and not pd.isna(error_ntp) else 0)
            + (1 * (error_rtp - base_error_rtp) / base_error_rtp if error_rtp is not None and not pd.isna(error_rtp) else 0)
        )
    )
    return (delta_m * 100).round(2)

In [116]:
# df_original = pd.read_csv("P2P/P2P_best_results.csv")
# df_original = pd.read_csv("Production/Production_best_results.csv")
df_original = pd.read_csv("HelpDesk/HelpDesk_best_results.csv")
DATASET = "Helpdesk"
df_original

,Model,MTL,Tasks,Learning Rate,MTL HPO,NEXT_ACTIVITY_mean,NEXT_ACTIVITY_std,NEXT_TIME_mean,NEXT_TIME_std,REMAINING_TIME_mean,REMAINING_TIME_std,Best Epoch_mean
0,LSTM,UW_SO,"('next_activity', 'next_time', 'remaining_time')",0.001,20.0,0.798867,0.004119,5.063433,0.115500,6.345700,0.116460,37.000000
1,LSTM,DWA,"('next_activity', 'next_time', 'remaining_time')",0.001,NaN,0.797933,0.006453,5.085300,0.144454,6.315100,0.097504,38.000000
2,LSTM,RLW,"('next_activity', 'next_time', 'remaining_time')",0.001,NaN,0.797367,0.005552,5.079100,0.086270,6.334767,0.062219,43.000000
3,LSTM,GradDrop,"('next_activity', 'next_time', 'remaining_time')",0.001,NaN,0.798067,0.005886,5.061933,0.095495,6.325167,0.079543,45.666667
4,LSTM,CAGrad,"('next_activity', 'next_time', 'remaining_time')",0.001,0.1,0.795300,0.001868,5.070100,0.138868,6.335200,0.104405,30.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
152,CNN,GradNorm,"('next_activity', 'next_time')",0.001,0.5,0.783533,0.000723,5.379233,0.165890,NaN,NaN,55.666667
153,CNN,GLS,"('next_activity', 'next_time')",0.001,NaN,0.782967,0.000551,5.386067,0.100665,NaN,NaN,45.666667
154,CNN,EW,"('next_activity',)",0.001,NaN,0.662700,0.007709,NaN,NaN,NaN,NaN,41.333333
155,CNN,EW,"('remaining_time',)",0.001,NaN,NaN,NaN,NaN,NaN,7.006833,0.113711,52.000000


In [ ]:
unique_task_combinations = list(df_original['Tasks'].unique())
# remove stl tasks
unique_task_combinations.remove("('next_activity',)")
unique_task_combinations.remove("('remaining_time',)")
unique_task_combinations.remove("('next_time',)")

nap_task = "('next_activity',)"
ntp_task = "('next_time',)"
rtp_task = "('remaining_time',)"
ts = [nap_task, ntp_task, rtp_task]
ts_names_mean = ['NEXT_ACTIVITY_mean', 'NEXT_TIME_mean', 'REMAINING_TIME_mean']
ts_names_std = ['NEXT_ACTIVITY_std', 'NEXT_TIME_std', 'REMAINING_TIME_std']

unique_architectures = df_original['Model'].unique()

for architecture in unique_architectures:
    print(f"architecture: {architecture}")
    # get STL results for this architecture
    stl_row = {'MTO': 'STL'}
    i = 0
    base_acc_nap = None
    base_error_ntp = None
    base_error_rtp = None

    for t in ts:
        filtered_df = df_original[(df_original['Tasks'] == t) & (df_original['Model'] == architecture)]
        mean = filtered_df[ts_names_mean[i]].values[0]
        std = filtered_df[ts_names_std[i]].values[0]
        if 'activity' in t:
            stl_row['NAP'] = f"{mean:.2f} ({std:.2f})"
            base_acc_nap = mean
        elif 'next_time' in t:
            stl_row['NTP'] = f"{mean:.2f} ({std:.2f})"
            base_error_ntp = mean
        else:
            stl_row['RTP'] = f"{mean:.2f} ({std:.2f})"
            base_error_rtp = mean
        i += 1
    stl_row['Best Epoch'] = "/"
    stl_row['$\Delta_m$'] = "/"

    # Apply row-wise and assign to the column
    df_original['$\Delta_m$'] = df_original.apply(
        lambda row: calc_delta_m(
            base_acc_nap=base_acc_nap,
            base_error_ntp=base_error_ntp,
            base_error_rtp=base_error_rtp,
            acc_nap=row['NEXT_ACTIVITY_mean'],
            error_ntp=row['NEXT_TIME_mean'],
            error_rtp=row['REMAINING_TIME_mean']
        ),
        axis=1
    )

    for task_combination in unique_task_combinations:
        df = df_original.copy()
        stl_row_copy = stl_row.copy()
        # rename some method names in the MTL column such as 'UW_SO' to 'UW-SO'
        df['MTL'] = df['MTL'].replace('UW_SO', 'UW-SO')
        df['MTL'] = df['MTL'].replace('UW_O', 'UW-O')
        df['MTL'] = df['MTL'].replace('Nash_MTL', 'NashMTL')

        print(f"task_combination: {task_combination}")
        task_combi = ""
        df = df[(df['Tasks'] == task_combination) & (df['Model'] == architecture)]
        # add new columns
        if not df['NEXT_ACTIVITY_mean'].isna().any():
            df['NAP'] = df.apply(
                lambda row: f"{row['NEXT_ACTIVITY_mean']:.2f} ({row['NEXT_ACTIVITY_std']:.2f})",
                axis=1
            )
            task_combi += "NAP+"
        else:
            # remove NAP in stl_row dict
            stl_row_copy.pop('NAP')
        if not df['NEXT_TIME_mean'].isna().any():
            df['NTP'] = df.apply(
                lambda row: f"{row['NEXT_TIME_mean']:.2f} ({row['NEXT_TIME_std']:.2f})",
                axis=1
            )
            task_combi += "NTP+"
        else:
            # remove NTP in stl_row dict
            stl_row_copy.pop('NTP')
        if not df['REMAINING_TIME_mean'].isna().any():
            df['RTP'] = df.apply(
                lambda row: f"{row['REMAINING_TIME_mean']:.2f} ({row['REMAINING_TIME_std']:.2f})",
                axis=1
            )
            task_combi += "RTP"
        else:
            # remove RTP in stl_row dict
            stl_row_copy.pop('RTP')
        if task_combi[-1] == "+":
            task_combi = task_combi[:-1]
        # rename best epoch column
        df = df.rename(columns={'Best Epoch_mean': 'Best Epoch'})
        df['Best Epoch'] = df.apply(
            lambda row: f"{row['Best Epoch']:.0f}",
            axis=1
        )

        df = df.rename(columns={'$\Delta_m$': 'Delta_m'})
        df['Delta_m'] = df.apply(
            lambda row: f"{row['Delta_m']:.2f}",
            axis=1
        )
        df = df.rename(columns={'Delta_m': '$\Delta_m$'})

        # rename MTL column
        df = df.rename(columns={'MTL': 'MTO'})
        # remove columns
        df = df.drop(columns=['Tasks', 'Model', 'Learning Rate', 'NEXT_ACTIVITY_mean', 'NEXT_TIME_mean', 'REMAINING_TIME_mean', 'NEXT_ACTIVITY_std', 'NEXT_TIME_std', 'REMAINING_TIME_std', 'MTL HPO'])

        # add STL results in one line at the top
        df = pd.concat([pd.DataFrame([stl_row_copy]), df])

        # reorder the rows depending on the MTO method such that the following order is respected:
        mto_order = ['STL', 'EW', 'Scalarization', 'DWA', 'RLW', 'UW', 'UW-O', 'UW-SO', 'GLS', 'IMTL', 'GradNorm', 'NashMTL', 'GradDrop', 'PCGrad', 'CAGrad']
        if 'Scalarization' not in df['MTO'].values:
            mto_order.remove('Scalarization')
        # reorder based on mto_order
        df = df.sort_values(by='MTO', key=lambda x: x.map(lambda y: mto_order.index(y)))



        latex_tabular = df.to_latex(
            index=False,
            bold_rows=False,
            column_format="lrrrrr",
        )

        # Wrap in table* and remove indentation
        latex_full = f"\\begin{{table*}}[htbp] \\caption{{Results on the {DATASET} dataset for {architecture} with task combination {task_combi}}} \\label{{tab:{DATASET}_results_{architecture}_{task_combi}}} \\centering {latex_tabular}\\end{{table*}}"

        print(latex_full)
        print("#################")


architecture: LSTM
task_combination: ('next_activity', 'next_time', 'remaining_time')
\begin{table*}[htbp] \caption{Results on the Helpdesk dataset for LSTM with task combination NAP+NTP+RTP} \label{tab:Helpdesk_results_LSTM_NAP+NTP+RTP} \centering \begin{tabular}{lrrrrr}
\toprule
MTO & NAP & NTP & RTP & Best Epoch & $\Delta_m$ \\
\midrule
STL & 0.68 (0.01) & 5.09 (0.10) & 6.38 (0.06) & / & / \\
EW & 0.80 (0.00) & 5.07 (0.15) & 6.33 (0.10) & 34 & -6.43 \\
DWA & 0.80 (0.01) & 5.09 (0.14) & 6.32 (0.10) & 38 & -6.39 \\
RLW & 0.80 (0.01) & 5.08 (0.09) & 6.33 (0.06) & 43 & -6.30 \\
UW & 0.80 (0.01) & 5.10 (0.13) & 6.34 (0.08) & 39 & -6.20 \\
UW-O & 0.80 (0.00) & 5.07 (0.12) & 6.37 (0.03) & 18 & -6.17 \\
UW-SO & 0.80 (0.00) & 5.06 (0.12) & 6.35 (0.12) & 37 & -6.42 \\
GLS & 0.80 (0.00) & 5.09 (0.10) & 6.35 (0.07) & 26 & -6.20 \\
IMTL & 0.80 (0.01) & 5.10 (0.09) & 6.35 (0.05) & 34 & -5.98 \\
GradNorm & 0.80 (0.00) & 5.06 (0.10) & 6.35 (0.04) & 43 & -6.34 \\
NashMTL & 0.80 (0.00) & 5.09 (0.12) 